<a href="https://colab.research.google.com/github/Nimishbansal-16/Rumour-Detection-in-Social-Network/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk import wordnet
from nltk.corpus import stopwords
from nltk import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df = pd.read_csv('CharlieNew30.csv')

In [ ]:
df.dropna(inplace=True)

In [ ]:
df

,tweet_source_id,tweet_source_text,uverified,ufollowers,ulisted,ustatuses,ufavorites,ageofacc,tweet_reaction_text,annotation
0,552821437678223361,b'Best response to #CharlieHebdo attack -- oth...,False,26094.0,982.0,11003.0,325.0,6.0,b'@actuary21c @jacobwe Lots to satirise there!',0
1,552821437678223361,b'Best response to #CharlieHebdo attack -- oth...,False,26094.0,982.0,11003.0,325.0,6.0,b'@UniversityWatc1 @jacobwe absolutely. And th...,0
2,552821437678223361,b'Best response to #CharlieHebdo attack -- oth...,False,26094.0,982.0,11003.0,325.0,6.0,b'@jacobwe Lol!',0
3,552821437678223361,b'Best response to #CharlieHebdo attack -- oth...,False,26094.0,982.0,11003.0,325.0,6.0,b'@popularcritic1 @jacobwe they can ridicule m...,0
4,552821437678223361,b'Best response to #CharlieHebdo attack -- oth...,False,26094.0,982.0,11003.0,325.0,6.0,"b""@nick_elt @jacobwe to u it is. I'm not relig...",0
...,...,...,...,...,...,...,...,...,...,...
10826,553152395371630592,b'This widely shared cartoon about the #Charli...,True,58928.0,846.0,4148.0,1411.0,2.0,b'@lydaten wat probeerde je te doen ;-)',1
10827,553152395371630592,b'This widely shared cartoon about the #Charli...,True,58928.0,846.0,4148.0,1411.0,2.0,b'@sven10hove laten zien ... Prachtige tekenin...,1
10828,553152395371630592,b'This widely shared cartoon about the #Charli...,True,58928.0,846.0,4148.0,1411.0,2.0,b'@lydaten :)',1
10829,553160652567498752,b'French police: Said Kouachi and Cherif Kouac...,True,3847705.0,37089.0,100712.0,560.0,6.0,b'\xe2\x80\x9c@ABC: French police: Said Kouach...,1


In [ ]:
df['tweet_reaction_text'].value_counts()

b'@guardiannews'                                                                                                                                                         7
b'@Telegraph'                                                                                                                                                            4
b'@ajplus'                                                                                                                                                               4
b'@Independent @DaveBrownToons'                                                                                                                                          4
b'@DavidClinchNews'                                                                                                                                                      3
                                                                                                                                                 

In [ ]:
import re
def clean_text(text):
  text = str(text[1:])
  text = re.sub('[^a-zA-Z]', " ", text) #remove punctuations and numbers
  text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text) # Single character removal
  text = re.sub(r'\s+', " ", text) #remove extra spaces
  return text

df['tweet_reaction_text'] = df['tweet_reaction_text'].apply(clean_text)
df['tweet_source_text'] = df['tweet_source_text'].apply(clean_text)

In [ ]:
df['len'] = df['tweet_reaction_text'].apply(lambda x: len(x))

In [ ]:
df = df[df['len']<140]
df.reset_index(inplace=True, drop=True)

In [ ]:
df

,tweet_source_id,tweet_source_text,uverified,ufollowers,ulisted,ustatuses,ufavorites,ageofacc,tweet_reaction_text,annotation,len
0,552821437678223361,Best response to CharlieHebdo attack other th...,False,26094.0,982.0,11003.0,325.0,6.0,actuary jacobwe Lots to satirise there,0,40
1,552821437678223361,Best response to CharlieHebdo attack other th...,False,26094.0,982.0,11003.0,325.0,6.0,UniversityWatc jacobwe absolutely And they kn...,0,129
2,552821437678223361,Best response to CharlieHebdo attack other th...,False,26094.0,982.0,11003.0,325.0,6.0,jacobwe Lol,0,13
3,552821437678223361,Best response to CharlieHebdo attack other th...,False,26094.0,982.0,11003.0,325.0,6.0,popularcritic jacobwe they can ridicule my re...,0,125
4,552821437678223361,Best response to CharlieHebdo attack other th...,False,26094.0,982.0,11003.0,325.0,6.0,nick elt jacobwe to it is m not religious but...,0,109
...,...,...,...,...,...,...,...,...,...,...,...
10603,553152395371630592,This widely shared cartoon about the CharlieH...,True,58928.0,846.0,4148.0,1411.0,2.0,lydaten wat probeerde je te doen,1,34
10604,553152395371630592,This widely shared cartoon about the CharlieH...,True,58928.0,846.0,4148.0,1411.0,2.0,sven hove laten zien Prachtige tekening Dacht...,1,54
10605,553152395371630592,This widely shared cartoon about the CharlieH...,True,58928.0,846.0,4148.0,1411.0,2.0,lydaten,1,9
10606,553160652567498752,French police Said Kouachi and Cherif Kouachi...,True,3847705.0,37089.0,100712.0,560.0,6.0,xe x ABC French police Said Kouachi and Cheri...,1,106


# **Bert Model**

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
X_input_ids = np.zeros((len(df), 280))
X_attn_masks = np.zeros((len(df), 280))

In [ ]:
X_input_ids.shape, X_attn_masks.shape

((10608, 280), (10608, 280))

In [ ]:
# X_input_ids[1, :] = X_attn_masks[1, :]
# X_input_ids[1, :]

In [ ]:
def generate_training_data(df, ids, masks, tokenizer):
    for i, (twet, res) in enumerate(zip(df['tweet_source_text'], df['tweet_reaction_text'])):
        #print(res)
        tokenized_text = tokenizer(
            twet,
            res,
            max_length=280, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks
    #return None


In [ ]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)
#generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

In [ ]:
X_input_ids.shape, X_attn_masks.shape

((10608, 280), (10608, 280))

In [ ]:
X_input_ids[3445]

array([  101.,  2256.,  4301.,  2024.,  2256.,  6956.,  2058.,  1999.,
        2605.,  2317.,  4580., 14056.,  2758.,  8299.,  2522.,  1051.,
       12322.,  2480.,  3723.,  2226.,  1047.,  2140.,  4918.,  5369.,
        2497.,  3527., 18699.,  4779.,  4523.,  2522.,  7020.,  1043.,
        5104.,  2094., 28816.,   102.,  2136.,  2102.,  9148.,  2923.,
        2480.,  2017.,  2128.,  2725.,  2009.,  2005.,  1996.,  2769.,
        2008.,  3100.,  3374.,  4287.,  2006.,  1037.,  2100.,  8516.,
        2863.,  9541.,   102.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
      

In [ ]:
labels = np.zeros((len(df), 3))
labels.shape

(10608, 3)

In [ ]:
labels[np.arange(len(df)), df['annotation'].values] = 1 # one-hot encoded target tensor
labels.shape

(10608, 3)

In [ ]:
# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading...
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data

<TakeDataset element_spec=(TensorSpec(shape=(280,), dtype=tf.float64, name=None), TensorSpec(shape=(280,), dtype=tf.float64, name=None), TensorSpec(shape=(3,), dtype=tf.float64, name=None))>

In [ ]:
def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [ ]:
dataset = dataset.map(SentimentDatasetMapFunction) # converting to required format for tensorflow dataset

In [ ]:
dataset = dataset.shuffle(10000).batch(8, drop_remainder=True) # batch size, drop any left out tensor

In [ ]:
p = 0.8
train_size = int((len(df)//8)*p) # for each 16 batch of data we will have len(df)//16 samples, take 80% of that for train.
print(train_size)

1060


In [ ]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)
print(train_dataset)
print(val_dataset)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(8, 280), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(8, 280), dtype=tf.float64, name=None)}, TensorSpec(shape=(8, 3), dtype=tf.float64, name=None))>
<SkipDataset element_spec=({'input_ids': TensorSpec(shape=(8, 280), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(8, 280), dtype=tf.float64, name=None)}, TensorSpec(shape=(8, 3), dtype=tf.float64, name=None))>


# **Model**

In [ ]:
from transformers import TFBertModel

In [ ]:
model1 = TFBertModel.from_pretrained('bert-base-uncased') # bert base model with pretrained weights

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(280,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(280,), name='attention_mask', dtype='int32')
#initiazed the bert with above 2 input layers
bert_embds = model1.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)

#2 RELU -> neuron activation. 
intermediate_layer = tf.keras.layers.Dense(128, activation='relu', name='intermediate_layer')(bert_embds)

#3 intermediate layer is passed for output . 3 is neuron-> positive,negative,neutral
output_layer = tf.keras.layers.Dense(3, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

sentiment_model1 = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model1.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 280)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 280)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 280,                                         

In [ ]:
!pip install keras_metrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import keras_metrics
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
print(keras_metrics.precision().name)
print(keras_metrics.recall().name)
print(keras_metrics.f1_score().name)
precision = keras_metrics.precision()
recall = keras_metrics.recall()
f1 = keras_metrics.f1_score()

precision
recall
f1_score


In [ ]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [ ]:
sentiment_model1.compile(optimizer=optim, loss=loss_func, metrics=[acc,precision,recall,f1])

In [ ]:
###charlie direct all charlie tweets and responses as input
hist = sentiment_model1.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=2
)

Epoch 1/2


/usr/local/lib/python3.7/dist-packages/keras_metrics/metrics.py:26: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return object.__getattribute__(self, name)


1060/1060 [==============================] - ETA: 0s - loss: 0.0984 - accuracy: 0.9646 - precision: 0.9270 - recall: 0.9422 - f1_score: 0.9329

1060/1060 [==============================] - 356s 324ms/step - loss: 0.0984 - accuracy: 0.9646 - precision: 0.9270 - recall: 0.9422 - f1_score: 0.9329 - val_loss: 0.0124 - val_accuracy: 0.9977 - val_precision: 0.9745 - val_recall: 0.9843 - val_f1_score: 0.9794
Epoch 2/2
1060/1060 [==============================] - 341s 322ms/step - loss: 0.0112 - accuracy: 0.9972 - precision: 0.9825 - recall: 0.9894 - f1_score: 0.9859 - val_loss: 0.0043 - val_accuracy: 0.9995 - val_precision: 0.9870 - val_recall: 0.9918 - val_f1_score: 0.9894


In [ ]:

#Now we want to get the predicted label for the tweet and response
#we will give input from CharlieTest.csv


In [ ]:
#Creating dataset from test dataframe
dftest = pd.read_csv('CharlieNew20.csv')
print(dftest)
dftest.dropna(inplace=True)
dftest['tweet_source_text'] = dftest['tweet_source_text'].apply(clean_text)
dftest['tweet_reaction_text'] = dftest['tweet_reaction_text'].apply(clean_text)
dftest['len'] = dftest['tweet_reaction_text'].apply(lambda x: len(x))

#why are we dropping text with greater than 140 length??
#df = df[df['len']<140]
#df.reset_index(inplace=True, drop=True)




         tweet_source_id                                  tweet_source_text  \
0     552784600502915072  b'Charlie Hebdo became well known for publishi...   
1     552784600502915072  b'Charlie Hebdo became well known for publishi...   
2     552784600502915072  b'Charlie Hebdo became well known for publishi...   
3     552784600502915072  b'Charlie Hebdo became well known for publishi...   
4     552784600502915072  b'Charlie Hebdo became well known for publishi...   
...                  ...                                                ...   
7214  552805488631758849  b"At least 12 killed in today's attack on sati...   
7215  552805488631758849  b"At least 12 killed in today's attack on sati...   
7216  552805488631758849  b"At least 12 killed in today's attack on sati...   
7217  552805488631758849  b"At least 12 killed in today's attack on sati...   
7218  552805488631758849  b"At least 12 killed in today's attack on sati...   

      uverified  ufollowers   ulisted  ustatuses  u

In [ ]:
#already executed#!pip install transformers
dftest

,tweet_source_id,tweet_source_text,uverified,ufollowers,ulisted,ustatuses,ufavorites,ageofacc,tweet_reaction_text,annotation,len
0,552784600502915072,Charlie Hebdo became well known for publishin...,True,41591.0,1657.0,15128.0,0.0,4.0,Now dead in shooting there today RT BBCDaniel...,0,136
1,552784600502915072,Charlie Hebdo became well known for publishin...,True,41591.0,1657.0,15128.0,0.0,4.0,BBCDanielS BBCWorld m guessing this is being ...,0,90
2,552784600502915072,Charlie Hebdo became well known for publishin...,True,41591.0,1657.0,15128.0,0.0,4.0,BBCDanielS BBCWorld why would you mention tha...,0,94
3,552784600502915072,Charlie Hebdo became well known for publishin...,True,41591.0,1657.0,15128.0,0.0,4.0,BBCDanielS BBCWorld perps identified,0,38
4,552784600502915072,Charlie Hebdo became well known for publishin...,True,41591.0,1657.0,15128.0,0.0,4.0,BBCDanielS BBCWorld who is charlie hebdo,0,42
...,...,...,...,...,...,...,...,...,...,...,...
7214,552805488631758849,At least killed in today attack on satirical ...,True,22667902.0,160810.0,37161.0,13.0,8.0,Chusid DaliaChai cnnbrk hehe Jews are good by...,1,130
7215,552805488631758849,At least killed in today attack on satirical ...,True,22667902.0,160810.0,37161.0,13.0,8.0,DaliaChai cnnbrk r not getting my point monst...,1,135
7216,552805488631758849,At least killed in today attack on satirical ...,True,22667902.0,160810.0,37161.0,13.0,8.0,Chusid DaliaChai cnnbrk look you fool we are ...,1,118
7217,552805488631758849,At least killed in today attack on satirical ...,True,22667902.0,160810.0,37161.0,13.0,8.0,hassanbasmer cnnbrk free speech They made fun...,1,124


In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
X_input_ids_test = np.zeros((len(dftest), 280))
X_attn_masks_test = np.zeros((len(dftest), 280))
X_input_ids_test.shape, X_attn_masks_test.shape

((7219, 280), (7219, 280))

In [ ]:
def generate_testing_data(df, ids, masks, tokenizer):
    for i, (twet, res) in enumerate(zip(df['tweet_source_text'], df['tweet_reaction_text'])):
        #print(res)
        tokenized_text = tokenizer(
            twet,
            res,
            max_length=280, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        #print(i)
        #print(res)
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [ ]:
X_input_ids_test, X_attn_masks_test = generate_testing_data(dftest, X_input_ids_test, X_attn_masks_test, tokenizer)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
#CSV's row number 2 is at the Oth index of numpyarray
X_input_ids_test[0]
X_attn_masks_test[0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [ ]:
type(X_input_ids_test)
type(X_attn_masks_test)

numpy.ndarray

In [ ]:
y_classes = sentiment_model1.predict([X_input_ids_test,X_attn_masks_test])

In [ ]:
y_pred_model = y_classes.argmax(axis=-1)
y_pred_model

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
np.savetxt("charlie_pred_label_new20.csv", y_pred_model,header="output_predicted", delimiter=",")

In [ ]:
##Appended this charlie_pred_label.csv  as the output_predicted column in the CharlieTest.csv manually
#EXECUTED THE FOLLOWING CODE
import csv
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix

dfmetrics = pd.read_csv('CharlieNew20.csv')
print(dfmetrics)
yactual = dfmetrics['annotation']
yactual = yactual.to_numpy()
dfmetricspred = pd.read_csv('charlie_pred_label_new20.csv')
print(dfmetricspred)
PredLabel = dfmetricspred['# output_predicted']
PredLabel = PredLabel.to_numpy()

print("Accuracy: " )
print(accuracy_score(yactual,PredLabel))

print("precision: ")
print(precision_score(yactual,PredLabel))

print("Recall:  ")
print( recall_score(yactual,PredLabel))

print("F1score: ")
print(f1_score(yactual,PredLabel))

print("Confusion Matrix: ")
print(confusion_matrix(yactual,PredLabel))

         tweet_source_id                                  tweet_source_text  \
0     552784600502915072  b'Charlie Hebdo became well known for publishi...   
1     552784600502915072  b'Charlie Hebdo became well known for publishi...   
2     552784600502915072  b'Charlie Hebdo became well known for publishi...   
3     552784600502915072  b'Charlie Hebdo became well known for publishi...   
4     552784600502915072  b'Charlie Hebdo became well known for publishi...   
...                  ...                                                ...   
7214  552805488631758849  b"At least 12 killed in today's attack on sati...   
7215  552805488631758849  b"At least 12 killed in today's attack on sati...   
7216  552805488631758849  b"At least 12 killed in today's attack on sati...   
7217  552805488631758849  b"At least 12 killed in today's attack on sati...   
7218  552805488631758849  b"At least 12 killed in today's attack on sati...   

      uverified  ufollowers   ulisted  ustatuses  u

In [ ]:
Model2Train50 = pd.concat([dftest, dfmetricspred], axis=1)

In [ ]:
Model2Train50.to_csv('CharlieModel1PredictedUnseen20.csv', index=False)

In [ ]:
Model2Train50

,tweet_source_id,tweet_source_text,uverified,ufollowers,ulisted,ustatuses,ufavorites,ageofacc,tweet_reaction_text,annotation,len,# output_predicted
0,552784600502915072,Charlie Hebdo became well known for publishin...,True,41591.0,1657.0,15128.0,0.0,4.0,Now dead in shooting there today RT BBCDaniel...,0,136,0.0
1,552784600502915072,Charlie Hebdo became well known for publishin...,True,41591.0,1657.0,15128.0,0.0,4.0,BBCDanielS BBCWorld m guessing this is being ...,0,90,0.0
2,552784600502915072,Charlie Hebdo became well known for publishin...,True,41591.0,1657.0,15128.0,0.0,4.0,BBCDanielS BBCWorld why would you mention tha...,0,94,0.0
3,552784600502915072,Charlie Hebdo became well known for publishin...,True,41591.0,1657.0,15128.0,0.0,4.0,BBCDanielS BBCWorld perps identified,0,38,0.0
4,552784600502915072,Charlie Hebdo became well known for publishin...,True,41591.0,1657.0,15128.0,0.0,4.0,BBCDanielS BBCWorld who is charlie hebdo,0,42,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7214,552805488631758849,At least killed in today attack on satirical ...,True,22667902.0,160810.0,37161.0,13.0,8.0,Chusid DaliaChai cnnbrk hehe Jews are good by...,1,130,1.0
7215,552805488631758849,At least killed in today attack on satirical ...,True,22667902.0,160810.0,37161.0,13.0,8.0,DaliaChai cnnbrk r not getting my point monst...,1,135,1.0
7216,552805488631758849,At least killed in today attack on satirical ...,True,22667902.0,160810.0,37161.0,13.0,8.0,Chusid DaliaChai cnnbrk look you fool we are ...,1,118,1.0
7217,552805488631758849,At least killed in today attack on satirical ...,True,22667902.0,160810.0,37161.0,13.0,8.0,hassanbasmer cnnbrk free speech They made fun...,1,124,1.0


In [ ]:
Model2Train50['annotation'] = Model2Train50['annotation'].astype('float')
#annotation = Model2Train50.groupby(['tweet_source_id'])['annotation'].mean()

In [ ]:
Model2Train50

,tweet_source_id,tweet_source_text,uverified,ufollowers,ulisted,ustatuses,ufavorites,ageofacc,tweet_reaction_text,annotation,len,# output_predicted
0,553111157738057728,So now let talk seriously about CharlieHebdo ...,False,15954.0,238.0,6556.0,128.0,2.0,janetravers It came moments after perfectly r...,0.0,107,0.0
1,553111157738057728,So now let talk seriously about CharlieHebdo ...,False,15954.0,238.0,6556.0,128.0,2.0,janetravers have no idea why you felt the nee...,0.0,115,0.0
2,553111157738057728,So now let talk seriously about CharlieHebdo ...,False,15954.0,238.0,6556.0,128.0,2.0,islandlayer xce xbc xcf xce xbf xcf xce xbf x...,0.0,481,0.0
3,553111157738057728,So now let talk seriously about CharlieHebdo ...,False,15954.0,238.0,6556.0,128.0,2.0,islandlayer xcf c xcf xce xb xce xb xce xad x...,0.0,457,0.0
4,553111157738057728,So now let talk seriously about CharlieHebdo ...,False,15954.0,238.0,6556.0,128.0,2.0,SikanasGiorgos xce xcf xcf xcf xce xb xce xbd...,0.0,646,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
18040,553591259672379392,Update PA gunman holding hostages in Paris gr...,True,1380592.0,14597.0,27260.0,0.0,6.0,SkyNewsBreak good shoot them all Paris will m...,1.0,59,1.0
18041,553591259672379392,Update PA gunman holding hostages in Paris gr...,True,1380592.0,14597.0,27260.0,0.0,6.0,SkyNewsBreak SteeryBlade GET IN Fucking lunat...,1.0,55,1.0
18042,553591259672379392,Update PA gunman holding hostages in Paris gr...,True,1380592.0,14597.0,27260.0,0.0,6.0,xe x SkyNewsBreak Update PA gunman holding ho...,1.0,101,1.0
18043,553591259672379392,Update PA gunman holding hostages in Paris gr...,True,1380592.0,14597.0,27260.0,0.0,6.0,SkyNewsBreak hope it was VERY painful Rot in ...,1.0,51,1.0


In [ ]:
Model2Train50.dtypes

tweet_source_id          int64
tweet_source_text       object
uverified                 bool
ufollowers             float64
ulisted                float64
ustatuses              float64
ufavorites             float64
ageofacc               float64
tweet_reaction_text     object
annotation             float64
len                      int64
# output_predicted     float64
dtype: object

In [ ]:
annotation = Model2Train50.groupby(['tweet_source_id'])['annotation']
annotation.head()


0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
18025    1.0
18026    1.0
18027    1.0
18028    1.0
18029    1.0
Name: annotation, Length: 4990, dtype: float64

In [ ]:
op = Model2Train50.groupby(['tweet_source_id'])['# output_predicted'].mean()


In [ ]:
print(op)

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
18025    1.0
18026    1.0
18027    1.0
18028    1.0
18029    1.0
Name: # output_predicted, Length: 4990, dtype: float64


In [ ]:
test = pd.concat([op], axis=1)
#print(test)
test.to_csv('testavg20.csv')
